# SQLite3 - Python

Code vu sur le diaporama.

- Création de la base de données, 

- Création de table `utilisateurs`, 

- Insertion de données, 

- Modification de connées, 

- Effacement d'entrées, 

- Sélection de données.

Exécutez ce code pour générer la base données avec une table utilisateurs.

In [1]:
import sqlite3

# Effacer le fichier de base de données s'il existe déjà
import os
if os.path.exists("database.db"):
    os.remove("database.db")

# Création / ouverture d'une base de données SQLite3
with sqlite3.connect('database.db') as conn:

    # écriture dans la base de données
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS utilisateurs (id INTEGER PRIMARY KEY AUTOINCREMENT, nom TEXT, email TEXT)")

    # ajouter des données dans la base de données
    cur.execute("INSERT INTO utilisateurs (nom, email) VALUES (?, ?)", ("Alice", "alice@ex.com"))

    # Insertion de données dans la table 'utilisateurs'
    nouvel_utilisateur = ("John Doe", "john.doe@example.com")
    cur.execute("INSERT INTO utilisateurs (nom, email) VALUES (?, ?)", nouvel_utilisateur)

    # Pour insérer plusieurs lignes en une seule requête, utilisez executemany
    nouveaux_utilisateurs = [("Jane Smith", "jane.smith@example.com"), ("Bob Johnson", "bob.johnson@example.com")]
    cur.executemany("INSERT INTO utilisateurs (nom, email) VALUES (?, ?)", nouveaux_utilisateurs)

    # Validez la transaction et enregistrez les modifications
    conn.commit()

    # Lire les données de la base de données
    cur.execute("SELECT * FROM utilisateurs")
    resultat = cur.fetchall()

    # Afficher les données
    print(" Après l'insertion des données : ".center(80, "-"))
    for ligne in resultat:
        print(ligne)

    # Mise à jour du champ 'email' pour un utilisateur spécifique
    utilisateur_id = 1
    nouvel_email = "nouveau.email.alice@example.com"
    cur.execute("UPDATE utilisateurs SET email = ? WHERE id = ?", (nouvel_email, utilisateur_id))

    # stopper la transaction et enregistrer les modifications dans la base de données
    conn.commit()

    # Suppression d'un utilisateur spécifique en fonction de son ID
    utilisateur_id = 4
    cur.execute("DELETE FROM utilisateurs WHERE id = ?", (utilisateur_id,))

    # stopper la transaction et enregistrer les modifications dans la base de données
    conn.commit()

    # Lire les données de la base de données
    cur.execute("SELECT * FROM utilisateurs")
    resultat = cur.fetchall()

    # Afficher les données
    print(" Après la mise à jour et la suppression des données : ".center(80, "-"))
    for ligne in resultat:
        print(ligne)

# fermer la connexion
# conn.close() non nécessaire car le 'with' s'en charge

----------------------- Après l'insertion des données : ------------------------
(1, 'Alice', 'alice@ex.com')
(2, 'John Doe', 'john.doe@example.com')
(3, 'Jane Smith', 'jane.smith@example.com')
(4, 'Bob Johnson', 'bob.johnson@example.com')
------------- Après la mise à jour et la suppression des données : -------------
(1, 'Alice', 'nouveau.email.alice@example.com')
(2, 'John Doe', 'john.doe@example.com')
(3, 'Jane Smith', 'jane.smith@example.com')


## Exercice 1 - Table `livres`

Ajoutez une table `livres` et une table `auteurs` à la base de données.

Ces tables contiendront respectivement les colonnes suivantes :

1. Livres :

    - `id` : identifiant integer clé primaire (autoincrementée).
    - `titre` : texte
    - `pitch` : texte (résumé)
    - `auteur_id` : clé étrangère vers la table `auteurs`
    - `date_public` : date du livre

.

2. Auteurs :

    - `id` : identifiant integer clé primaire (autoincrementée).
    - `nom_auteur` : nom complet de l'auteur


Vous devez faire le script python pour lire le fichier JSON (ou un autre si besoin) et automatiquement générer les deux tables citées.

Cette procédure automatique doit pouvoir aussi être appelée si on donne un fichier JSON pour `ajouter` des données à la base de données. On considérera que dans ce cas le nom d'un auteur est un identifiant unique (i.e. s'il est déjà dans la table, on n'ajoute pas un nouvel auteur), et de même que le titre d'un livre est unique.

Pour contrôler la bonne exécution du code, pensez à faire quelques affichages voire à générer un fichier log.txt :

```
nouvel auteur Victor Hugo ajouté id = 1
nouvel auteur Gustave Flaubert ajouté id = 2
nouvel auteur Émile Zola ajouté id = 3
nouvel auteur Albert Camus ajouté id = 4
auteur Albert Camus déjà existant
nouvel auteur Charles Baudelaire ajouté id = 5
nouvel auteur Pierre Choderlos de Laclos ajouté id = 6
nouvel auteur Louis-Ferdinand Céline ajouté id = 7
nouvel auteur Georges Perec ajouté id = 8
nouvel auteur Khaled Hosseini ajouté id = 9
nouvel auteur Umberto Eco ajouté id = 10
nouvel auteur Franz Kafka ajouté id = 11
auteur Émile Zola déjà existant
nouvel auteur Alain Damasio ajouté id = 12
nouvel auteur Maurice Druon ajouté id = 13
nouvel auteur Alexandre Dumas ajouté id = 14
nouvel auteur Léon Tolstoï ajouté id = 15
auteur Alexandre Dumas déjà existant
nouvel auteur Patrick Süskind ajouté id = 16
auteur Georges Perec déjà existant
nouvel auteur Stendhal ajouté id = 17
nouvel auteur Guy de Maupassant ajouté id = 18
auteur Émile Zola déjà existant
nouvel auteur Alain-Fournier ajouté id = 19
auteur Gustave Flaubert déjà existant
```


In [2]:
import sqlite3
import json

def main():
    # Connexion à la base de données
    with sqlite3.connect('database.db') as conn:
        cur = conn.cursor()

        # Création des tables Auteurs et Livres
        cur.execute("""
            CREATE TABLE IF NOT EXISTS Auteurs (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nom_auteur TEXT UNIQUE
            )
        """)
        cur.execute("""
            CREATE TABLE IF NOT EXISTS Livres (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                titre TEXT UNIQUE,
                pitch TEXT,
                date_public DATE,
                auteur_id INTEGER,
                FOREIGN KEY (auteur_id) REFERENCES Auteurs(id)
            )
        """)

        # Chargement des données depuis le fichier JSON
        with open('data_books.json') as file:
            data = json.load(file)

        # Ajout des auteurs si nécessaires
        for book in data:
            nom_auteur = book['author']
            cur.execute("INSERT OR IGNORE INTO Auteurs (nom_auteur) VALUES (?)", (nom_auteur,))
        
        # Ajout des livres
        for book in data:
            cur.execute("SELECT id FROM Auteurs WHERE nom_auteur = ?", (book['author'],))
            auteur_id = cur.fetchone()[0]  # Récupération de l'id de l'auteur
            
            cur.execute("""
                INSERT OR IGNORE INTO Livres (titre, pitch, date_public, auteur_id)
                VALUES (?, ?, ?, ?)
            """, (book['title'], book['content'], book['date'], auteur_id))

        # Affichage des données des tables
        cur.execute("SELECT * FROM Auteurs")
        print("Auteurs:")
        for row in cur.fetchall():
            print(row)

        cur.execute("SELECT * FROM Livres")
        print("\nLivres:")
        for row in cur.fetchall():
            print(row)

if __name__ == "__main__":
    main()


Auteurs:
(1, 'Victor Hugo')
(2, 'Gustave Flaubert')
(3, 'Émile Zola')
(4, 'Albert Camus')
(6, 'Charles Baudelaire')
(7, 'Pierre Choderlos de Laclos')
(8, 'Louis-Ferdinand Céline')
(9, 'Georges Perec')
(10, 'Khaled Hosseini')
(11, 'Umberto Eco')
(12, 'Franz Kafka')
(14, 'Alain Damasio')
(15, 'Maurice Druon')
(16, 'Alexandre Dumas')
(17, 'Léon Tolstoï')
(19, 'Patrick Süskind')
(21, 'Stendhal')
(22, 'Guy de Maupassant')
(24, 'Alain-Fournier')

Livres:
(1, 'Les Misérables', "L'histoire de Jean Valjean, un ancien bagnard en quête de rédemption, et de sa relation avec l'inspecteur Javert.", '14/04/1862', 1)
(2, 'Madame Bovary', "L'histoire d'Emma Bovary, une femme insatisfaite de sa vie conjugale et en quête de passion et de bonheur.", '01/10/1856', 2)
(3, 'Germinal', "L'histoire d'une grève des mineurs dans le nord de la France et de leur lutte pour de meilleures conditions de travail.", '28/11/1885', 3)
(4, "L'Étranger", "L'histoire de Meursault, un homme indifférent à la vie et à la mort,

## Exercice 2 - Ajouter colonne `emprunteur_id`

Ajouter une colonne `emprunteur_id` à la table `livres`. Ce sera une clé étrangère reliée à la table `utilisateurs`.

Voir les ordres `SQL` sur la doc : [https://www.sqlitetutorial.net/sqlite-cheat-sheet/](https://www.sqlitetutorial.net/sqlite-cheat-sheet/)

Générez de 1 à 4 livres empruntés (au hasard) pour chaque utilisateur.

Pensez à générer des affichages ou un log.txt pour vérifier l'exécution de vos scripts.

```
livre La Peste emprunté par Alice
livre L'Assommoir emprunté par John Doe
livre La Métamorphose emprunté par John Doe
livre Germinal emprunté par John Doe
livre Les Trois Mousquetaires emprunté par John Doe
livre La Guerre et la Paix emprunté par Jane Smith
livre Les Rois maudits emprunté par Jane Smith
livre Le Rouge et le Noir emprunté par Jane Smith
```


In [3]:
import sqlite3
import random

def main():
    with sqlite3.connect('database.db') as conn:
        cur = conn.cursor()

        # Étape 1 : Ajouter une colonne `emprunteur_id` à la table Livres (si elle n'existe pas déjà)
        try:
            cur.execute("ALTER TABLE Livres ADD COLUMN emprunteur_id INTEGER")
        except sqlite3.OperationalError:
            pass  # Colonne déjà ajoutée

        # Étape 2 : Ajouter une colonne `livres_empruntes` à la table Utilisateurs (si elle n'existe pas déjà)
        try:
            cur.execute("ALTER TABLE utilisateurs ADD COLUMN livres_empruntes INTEGER DEFAULT 0")
        except sqlite3.OperationalError:
            pass  # Colonne déjà ajoutée

        # Étape 3 : Récupérer les utilisateurs et leurs emprunts actuels
        cur.execute("SELECT id, livres_empruntes FROM utilisateurs")
        utilisateurs = [(row[0], row[1]) for row in cur.fetchall()]  # [(id, livres_empruntes), ...]

        # Étape 4 : Récupérer tous les livres non encore empruntés
        cur.execute("SELECT id FROM Livres WHERE emprunteur_id IS NULL")
        livres_disponibles = [row[0] for row in cur.fetchall()]  # Liste des IDs des livres disponibles

        # Étape 5 : Assigner les livres aux utilisateurs
        associations = []
        while livres_disponibles and utilisateurs:
            livre_id = livres_disponibles.pop()  # Prendre un livre
            utilisateur = random.choice(utilisateurs)  # Choisir un utilisateur au hasard
            utilisateur_id, livres_empruntes = utilisateur

            if livres_empruntes < 4:  # Limiter à 4 livres par utilisateur
                # Mettre à jour l'emprunteur pour ce livre
                cur.execute("UPDATE Livres SET emprunteur_id = ? WHERE id = ?", (utilisateur_id, livre_id))
                associations.append((livre_id, utilisateur_id))

                # Mettre à jour le nombre de livres empruntés
                livres_empruntes += 1
                cur.execute("UPDATE utilisateurs SET livres_empruntes = ? WHERE id = ?", (livres_empruntes, utilisateur_id))

                # Mettre à jour la liste des utilisateurs
                utilisateurs = [(uid, emprunts) for uid, emprunts in utilisateurs if uid != utilisateur_id] + [(utilisateur_id, livres_empruntes)]

            # Si un utilisateur atteint la limite, le retirer temporairement
            utilisateurs = [(uid, emprunts) for uid, emprunts in utilisateurs if emprunts < 4]

        conn.commit()

        # Étape 6 : Afficher les résultats et les enregistrer dans un fichier log.txt
        with open("log.txt", "w") as log_file:
            log_file.write("Associations livres - emprunteurs :\n")
            for livre_id, emprunteur_id in associations:
                cur.execute("SELECT titre FROM Livres WHERE id = ?", (livre_id,))
                livre_titre = cur.fetchone()[0]

                cur.execute("SELECT nom FROM utilisateurs WHERE id = ?", (emprunteur_id,))
                emprunteur_nom = cur.fetchone()[0]

                log_line = f"Livre '{livre_titre}' emprunté par {emprunteur_nom}\n"
                print(log_line.strip())
                log_file.write(log_line)

            log_file.write("\nNombre de livres empruntés par utilisateur :\n")
            cur.execute("SELECT nom, livres_empruntes FROM utilisateurs")
            for nom, livres_empruntes in cur.fetchall():
                log_line = f"{nom} a emprunté {livres_empruntes} livres\n"
                print(log_line.strip())
                log_file.write(log_line)

if __name__ == "__main__":
    main()


Livre 'Le Grand Meaulnes' emprunté par John Doe
Livre 'L'Assommoir' emprunté par Alice
Livre 'Bel-Ami' emprunté par Alice
Livre 'Le Rouge et le Noir' emprunté par Alice
Livre 'La Vie mode d'emploi' emprunté par John Doe
Livre 'Le Parfum' emprunté par Alice
Livre 'Les Trois Mousquetaires' emprunté par John Doe
Livre 'La Guerre et la Paix' emprunté par Jane Smith
Livre 'Le Comte de Monte-Cristo' emprunté par Jane Smith
Livre 'Les Rois maudits' emprunté par John Doe
Livre 'La Horde du Contrevent' emprunté par Jane Smith
Livre 'Au bonheur des dames' emprunté par Jane Smith
Alice a emprunté 4 livres
John Doe a emprunté 4 livres
Jane Smith a emprunté 4 livres


## Exercice 3 - Génération d'API

La base de données est finie !

Maintenant on souhaite créer en Flask / Jinja2 une API ReST pour cette table.

Faites l'API avec les "endpoints" suivants :

1. Chemins `/utilisateurs`, `/livres`, `/auteurs` : renvoient la liste en JSON des tables complètes

1. Chemin `/utilisateur/<utilisateur>` : renvoie le dictionnaire correspondant à l'utilisateur d'id `utilisateur` ou par l'utilisateur de nom `utilisateur` si un seul utilisateur porte ce nom-là. Si plusieurs portent le même nom, une erreur est renvoyée.

1. Chemin `/utilisateur/emprunts/<utilisateur>` : renvoie la liste des livres empruntés par l'utilisateur d'id `utilisateur` ou par l'utilisateur de nom `utilisateur` si un seul utilisateur porte ce nom-là.

1. Chemin `/livres/siecle/<numero>` : renvoie la liste des livres du siècle marqué.

1. Chemin `/livres/ajouter` : en POST ajoute un livre au format identique au fichier JSON (si l'auteur n'existe pas encore il est ajouté)

1. Chemin `/utilisateur/ajouter` : en POST ajoute un utilisateur (format {"nom": nom_user, "email": email_user})

1. Chemin `/utilisateur/<utilisateur>/supprimer` : en DELETE

1. Chemin `/utilisateur/{utilisateur_id}/emprunter/{livre_id}` : en PUT, permet d'emprunter un livre

1. Chemin `/utilisateur/{utilisateur_id}/rendre/{livre_id}` : en PUT, permet de rendre un livre


In [1]:
from flask import Flask, jsonify, request, abort, render_template
import sqlite3

app = Flask(__name__)

def execute_query(query, params=(), fetchone=False, commit=False):
    with sqlite3.connect("database.db") as conn:
        cur = conn.cursor()
        cur.execute(query, params)
        if commit:
            conn.commit()
            return cur.lastrowid
        if fetchone:
            return cur.fetchone()
        return cur.fetchall()

@app.route('/utilisateurs', methods=['GET'])
def get_utilisateurs():
    utilisateurs = execute_query("SELECT * FROM utilisateurs")
    return jsonify([{"id": u[0], "nom": u[1], "email": u[2], "livres_empruntes": u[3]} for u in utilisateurs])

@app.route('/livres', methods=['GET'])
def get_livres():
    livres = execute_query("SELECT * FROM Livres")
    return jsonify([{
        "id": l[0],
        "titre": l[1],
        "pitch": l[2],
        "date_public": l[3],
        "auteur_id": l[4],
        "emprunteur_id": l[5]
    } for l in livres])

@app.route('/auteurs', methods=['GET'])
def get_auteurs():
    auteurs = execute_query("SELECT * FROM Auteurs")
    return jsonify([{"id": a[0], "nom_auteur": a[1]} for a in auteurs])

@app.route('/utilisateur/<utilisateur>', methods=['GET'])
def get_utilisateur(utilisateur):
    if utilisateur.isdigit():
        result = execute_query("SELECT * FROM utilisateurs WHERE id = ?", (int(utilisateur),), fetchone=True)
    else:
        result = execute_query("SELECT * FROM utilisateurs WHERE nom = ?", (utilisateur,))
        if len(result) > 1:
            abort(400, "Plusieurs utilisateurs portent ce nom.")
        elif result:
            result = result[0]
    
    if result:
        return jsonify({"id": result[0], "nom": result[1], "email": result[2], "livres_empruntes": result[3]})
    abort(404, "Utilisateur non trouvé.")

@app.route('/utilisateur/emprunts/<utilisateur>', methods=['GET'])
def get_emprunts(utilisateur):
    if utilisateur.isdigit():
        utilisateur_id = int(utilisateur)
    else:
        user = execute_query("SELECT id FROM utilisateurs WHERE nom = ?", (utilisateur,), fetchone=True)
        if user:
            utilisateur_id = user[0]
        else:
            abort(404, "Utilisateur non trouvé.")

    livres = execute_query("SELECT titre FROM Livres WHERE emprunteur_id = ?", (utilisateur_id,))
    return jsonify([{"titre": livre[0]} for livre in livres])

@app.route('/livres/siecle/<int:numero>', methods=['GET'])
def get_livres_par_siecle(numero):
    start_year = (numero - 1) * 100 + 1
    end_year = start_year + 99           
    livres = execute_query(
        """
        SELECT id, titre, pitch, date_public, auteur_id, emprunteur_id
        FROM Livres
        WHERE CAST(SUBSTR(date_public, 7, 4) AS INTEGER) BETWEEN ? AND ?
        """,
        (start_year, end_year)
    )
    return jsonify([
        {
            "id": livre[0],
            "titre": livre[1],
            "pitch": livre[2],
            "date_public": livre[3],
            "auteur_id": livre[4],
            "emprunteur_id": livre[5]
        } for livre in livres
    ])
# Route pour afficher le formulaire HTML
@app.route('/utilisateur/ajouter', methods=['GET'])
def afficher_formulaire_ajouter_utilisateur():
    return render_template('ajouter_utilisateur.html')  # Cette ligne va chercher le fichier HTML dans le dossier templates

@app.route('/utilisateur/ajouter', methods=['POST'])
def ajouter_utilisateur():
    # Récupérer les données du formulaire avec request.form
    nom = request.form.get("nom")
    email = request.form.get("email")
    
    if not nom or not email:
        abort(400, "Données invalides.")
    
    utilisateur_id = execute_query(
        "INSERT INTO utilisateurs (nom, email) VALUES (?, ?)", 
        (nom, email), 
        commit=True
    )
    return jsonify({"id": utilisateur_id, "message": "Utilisateur ajouté avec succès."}), 201

#a partire de la pas verif

@app.route('/livres/ajouter', methods=['POST'])
def ajouter_livre():
    # Récupérer les données du livre via request.form ou request.json
    titre = request.json.get("titre")
    pitch = request.json.get("pitch")
    date_public = request.json.get("date_public")
    auteur_nom = request.json.get("auteur")  # Nom de l'auteur

    if not titre or not pitch or not date_public or not auteur_nom:
        abort(400, "Données invalides.")

    # Vérifier si l'auteur existe, sinon l'ajouter
    auteur = execute_query("SELECT id FROM Auteurs WHERE nom_auteur = ?", (auteur_nom,), fetchone=True)
    if not auteur:
        # Ajouter l'auteur s'il n'existe pas
        auteur_id = execute_query("INSERT INTO Auteurs (nom_auteur) VALUES (?)", (auteur_nom,), commit=True)
    else:
        auteur_id = auteur[0]

    # Ajouter le livre
    livre_id = execute_query(
        "INSERT INTO Livres (titre, pitch, date_public, auteur_id) VALUES (?, ?, ?, ?)",
        (titre, pitch, date_public, auteur_id),
        commit=True
    )

    return jsonify({"id": livre_id, "message": "Livre ajouté avec succès."}), 201

@app.route('/utilisateur/<utilisateur>/supprimer', methods=['DELETE'])
def supprimer_utilisateur(utilisateur):
    if utilisateur.isdigit():
        utilisateur_id = int(utilisateur)
    else:
        utilisateur = execute_query("SELECT id FROM utilisateurs WHERE nom = ?", (utilisateur,), fetchone=True)
        if not utilisateur:
            abort(404, "Utilisateur non trouvé.")
        utilisateur_id = utilisateur[0]

    # Supprimer l'utilisateur
    execute_query("DELETE FROM utilisateurs WHERE id = ?", (utilisateur_id,), commit=True)
    return jsonify({"message": "Utilisateur supprimé avec succès."})

@app.route('/utilisateur/<int:utilisateur_id>/emprunter/<int:livre_id>', methods=['PUT'])
def emprunter_livre(utilisateur_id, livre_id):
    # Vérifier si le livre existe et n'est pas déjà emprunté
    livre = execute_query("SELECT emprunteur_id FROM Livres WHERE id = ?", (livre_id,), fetchone=True)
    if not livre:
        abort(404, "Livre non trouvé.")
    
    if livre[0]:
        abort(400, "Ce livre est déjà emprunté.")

    # Mettre à jour le livre pour l'emprunt
    execute_query("UPDATE Livres SET emprunteur_id = ? WHERE id = ?", (utilisateur_id, livre_id), commit=True)

    return jsonify({"message": "Livre emprunté avec succès."})

@app.route('/utilisateur/<int:utilisateur_id>/rendre/<int:livre_id>', methods=['PUT'])
def rendre_livre(utilisateur_id, livre_id):
    # Vérifier si le livre est emprunté par l'utilisateur
    livre = execute_query("SELECT emprunteur_id FROM Livres WHERE id = ?", (livre_id,), fetchone=True)
    if not livre:
        abort(404, "Livre non trouvé.")
    
    if livre[0] != utilisateur_id:
        abort(400, "Ce livre n'a pas été emprunté par cet utilisateur.")

    # Mettre à jour le livre pour le rendre disponible
    execute_query("UPDATE Livres SET emprunteur_id = NULL WHERE id = ?", (livre_id,), commit=True)

    return jsonify({"message": "Livre rendu avec succès."})



if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5007, debug=False)




 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5007
Press CTRL+C to quit


## Exercice 4 - Swagger (OpenAPI documentation)

Ecrivez un fichier "Swagger" pour l'API : "openapi.json"

Ecrivez les deux premiers "endpoints" de l'API dans un fichier compatible OpenAPI 3.1


In [1]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, HTTPException, Form, Depends
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import sqlite3
from typing import List, Optional

app = FastAPI()

class Utilisateur(BaseModel):
    id: int
    nom: str
    email: str
    livres_empruntes: Optional[int] = 0

class Livre(BaseModel):
    id: int
    titre: str
    pitch: str
    date_public: str
    auteur_id: int
    emprunteur_id: Optional[int] = None

class Auteur(BaseModel):
    id: int
    nom_auteur: str

def execute_query(query, params=(), fetchone=False, commit=False):
    with sqlite3.connect("database.db") as conn:
        cur = conn.cursor()
        cur.execute(query, params)
        if commit:
            conn.commit()
            return cur.lastrowid
        if fetchone:
            return cur.fetchone()
        return cur.fetchall()

@app.get("/utilisateurs", response_model=List[Utilisateur])
async def get_utilisateurs():
    utilisateurs = execute_query("SELECT * FROM utilisateurs")
    return [{"id": u[0], "nom": u[1], "email": u[2], "livres_empruntes": u[3]} for u in utilisateurs]

@app.get("/livres", response_model=List[Livre])
async def get_livres():
    livres = execute_query("SELECT * FROM Livres")
    return [{
        "id": l[0],
        "titre": l[1],
        "pitch": l[2],
        "date_public": l[3],
        "auteur_id": l[4],
        "emprunteur_id": l[5]
    } for l in livres]

@app.get("/auteurs", response_model=List[Auteur])
async def get_auteurs():
    auteurs = execute_query("SELECT * FROM Auteurs")
    return [{"id": a[0], "nom_auteur": a[1]} for a in auteurs]

@app.get("/utilisateur/{utilisateur}", response_model=Utilisateur)
async def get_utilisateur(utilisateur: str):
    # Si l'utilisateur est un ID (entier)
    if utilisateur.isdigit():
        result = execute_query("SELECT * FROM utilisateurs WHERE id = ?", (int(utilisateur),), fetchone=True)
    else:
        # Recherche partielle par nom avec LIKE
        result = execute_query("SELECT * FROM utilisateurs WHERE nom LIKE ?", (f"%{utilisateur}%",), fetchone=True)
        if not result:
            raise HTTPException(status_code=404, detail="Utilisateur non trouvé.")
    
    if result:
        return {"id": result[0], "nom": result[1], "email": result[2], "livres_empruntes": result[3]}
    raise HTTPException(status_code=404, detail="Utilisateur non trouvé.")


@app.get("/utilisateur/emprunts/{utilisateur}", response_model=List[dict])
async def get_emprunts(utilisateur: str):
    # Si l'utilisateur est un ID (entier)
    if utilisateur.isdigit():
        utilisateur_id = int(utilisateur)
    else:
        # Recherche partielle par nom avec LIKE
        result = execute_query("SELECT id FROM utilisateurs WHERE nom LIKE ?", (f"%{utilisateur}%",), fetchone=True)
        if not result:
            raise HTTPException(status_code=404, detail="Utilisateur non trouvé.")
        utilisateur_id = result[0]
    
    # Récupérer les titres des livres empruntés par l'utilisateur
    livres = execute_query("SELECT titre FROM Livres WHERE emprunteur_id = ?", (utilisateur_id,))
    return [{"titre": livre[0]} for livre in livres]


@app.get("/livres/siecle/{numero}", response_model=List[Livre])
async def get_livres_par_siecle(numero: int):
    start_year = (numero - 1) * 100 + 1
    end_year = start_year + 99
    livres = execute_query(
        """
        SELECT id, titre, pitch, date_public, auteur_id, emprunteur_id
        FROM Livres
        WHERE CAST(SUBSTR(date_public, 7, 4) AS INTEGER) BETWEEN ? AND ?
        """,
        (start_year, end_year)
    )
    return [{
        "id": livre[0],
        "titre": livre[1],
        "pitch": livre[2],
        "date_public": livre[3],
        "auteur_id": livre[4],
        "emprunteur_id": livre[5]
    } for livre in livres]


@app.post("/utilisateur/ajouter")
async def ajouter_utilisateur(nom: str = Form(...), email: str = Form(...)):
    if not nom or not email:
        raise HTTPException(status_code=400, detail="Données invalides.")
    
    utilisateur_id = execute_query(
        "INSERT INTO utilisateurs (nom, email) VALUES (?, ?)", 
        (nom, email), 
        commit=True
    )
    return JSONResponse(content={"id": utilisateur_id, "message": "Utilisateur ajouté avec succès."}, status_code=201)

import uvicorn
uvicorn.run(app, host="0.0.0.0", port=5007)


INFO:     Started server process [7660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5007 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7660]
